<a href="https://colab.research.google.com/github/SirSirocco/DataScience_2025_1/blob/main/ENG4502_experiment_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEPENDÊNCIAS GERAIS

In [279]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
from itertools import product
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# FUNÇÕES

## Utilitário

In [231]:
def flatten_dict(dictionary: dict, fields_to_flatten: list[str]) -> dict:
    """
    Desestrutura os campos selecionados de um dicionário, retornando apenas os atributos desejados.

    :param dictionary: Dicionário original que contém os campos a serem desestruturados.
    :param fields_to_flatten: Lista de nomes dos campos que devem ser desestruturados (flatten).

    :return: Novo dicionário contendo apenas os atributos dos campos selecionados, combinados em um nível.
    """
    dict_flat = dict()

    for field in fields_to_flatten:
        value = dictionary.get(field)
        if isinstance(value, dict):
            dict_flat.update(value)

    return dict_flat

## Exibição

In [232]:
def df_show_domain(df: pd.DataFrame, columns: list[str]) -> None:
    """
    Exibe o tipo e os valores possíveis de columns segundo a ordem da lista de
    colunas.

    :param df: DataFrame a ter os domínios exibidos.
    :param columns: Lista dos nomes das colunas a serem exibidas.
    :return: None
    """
    for col in columns:
        print(f"Coluna:   {col}")
        print(f"dtype:    {df[col].dtype}")
        print(f"Domínio:  {df[col].unique()}\n")

def df_show_head(df: pd.DataFrame, n: int = 5) -> None:
    display(df.head(n))
    print(f"Shape: {df.shape}")

def df_show_null(df: pd.DataFrame) -> None:
    display(df.isnull().sum())

## Métricas

In [233]:
def efficiency_score(y_true: list[float], y_pred: list[float], average: bool = True) -> float:
    prec = precision_score(y_true, y_pred, average=None)
    rec = recall_score(y_true, y_pred, average=None)

    length = len(prec)
    if length == 0 or length != len(rec):
        raise ValueError("precision_score and recall_score must have the same length, which has to be greater than zero")

    eff = [(prec[i] + rec[i]) / 2.0 for i in range(length)]

    return np.mean(eff) if average else eff

def get_metrics_classification(y_true: pd.Series, y_pred: pd.Series, classes: list[str], view: bool = False) -> dict:
    """
    Exibe métricas de avaliação do modelo se view for True e as retorna
    em um dicionário. Se a métrica for atrelada à classe,
    a chave do dicionário terá a forma "<metrica>_<classe>".
    """
    # Constantes auxiliares
    DICT_METRICS_GENERAL = {
        "Acurácia": accuracy_score,
    }
    DICT_METRICS_CLASS = {
        "Precisão": precision_score,
        "Sensibilidade": recall_score,
        "F1-Score": f1_score,
        "Eficiência": efficiency_score,
    }

    metrics_general = dict()
    metrics_class = dict()
    dict_metric = dict()
    format_classes = [x.replace(" ", "_") for x in classes]
    num_classes = len(classes)

    # Computa métricas globais
    for metric, function in DICT_METRICS_GENERAL.items():
        metrics_general[metric] = dict_metric[metric] = (function(y_true, y_pred))

    # Computa métricas por classe
    for metric, function in DICT_METRICS_CLASS.items():
        metrics_class[metric] = function(y_true, y_pred, average=None)
        for i in range(num_classes):
            dict_metric[f"{metric}_{format_classes[i]}"] = metrics_class[metric][i]

    if view: # Exibe
        for metric, value in metrics_general.items():
            print(f"{metric}: %.4f" % value)

        for i in range(num_classes):
            print(f"{format_classes[i]}:")

            for metric, values in metrics_class.items():
                print(f"\t{metric}: %.4f" % values[i])

        # Adiciona respiro visual
        print("")

    return dict_metric

## Validação

In [254]:
def is_valid_series(s: pd.Series):
    return s is not None and (not s.empty) and s.notnull().all() and pd.api.types.is_numeric_dtype(s)

## Pré-Processamento

In [236]:
def preprocessing_map(df: pd.DataFrame, features: list[str], maps: list[dict]) -> pd.DataFrame:
    """
    Aplica mapeamentos personalizados a múltiplas colunas de um DataFrame.

    :param df: DataFrame original com os dados a serem transformados.
    :param features: Lista de nomes de colunas que serão mapeadas.
    :param maps: Lista de dicionários contendo os mapeamentos a serem aplicados, um para cada coluna,
                 respeitando a ordem das colunas em features.

    :return: Novo DataFrame com os mapeamentos aplicados nas colunas especificadas.
    """
    df_copy = df.copy()

    for feature, feature_map in zip(features, maps):
        df_copy[feature] = df[feature].map(feature_map)

    return df_copy

def subs_na_mean(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    df_copy = df.copy()
    for col in columns:
        mean = df[col].mean()
        df_copy[col] = df[col].fillna(mean)
    return df_copy

def subs_outliers(df: pd.DataFrame, columns: list[str], view:bool = False) -> pd.DataFrame:
    """
    Substitui outliers pela média. Se view for True, exibe
    dois gráficos para cada coluna: antes e depois da mudança.
    """

    # Guarda resultado anterior
    df_before = df.copy()

    for column in columns:
        # Obtém quartis 1 e 3
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)

        # Obtém IQR
        iqr = q3 - q1

        """
        Outlier se em (-inf, Q1 - 1.5 * IQR) ou (Q3 + 1.5 * IQR, +inf).
        Subsititui outliers por NA ('not available') segundo método IQR (InterQuartile Range).
        """
        df.loc[(df[column] < q1 - 1.5 * iqr) | (df[column] > q3 + 1.5 * iqr), column] = pd.NA

        # Substitui NAs pela média
        df[column] = df[column].fillna(df[column].mean())

    if view:
        for column in columns:
            # Exibe resultado inicial
            plt.boxplot(df_before[column])
            plt.title(f"ANTES: {column}")
            plt.show()

            # Exibe novo resultado
            plt.boxplot(df[column])
            plt.title(f"DEPOIS: {column}")
            plt.show()

    return df

## Casos de pré-processamento

In [237]:
def apply_feature_engineering(df, X, y, feat_eng, prefix, radixes):
    key = prefix
    if is_valid_series(feat_eng):
        df = pd.concat([df, feat_eng], axis=1)
        key += f"_{feat_eng.name}"
        feat_name = feat_eng.name
    else:
        key += radixes["absent"]
        feat_name = None
    return df, X, y, key, {"feat_eng": feat_name}

def apply_outliers(df, X, y, out, prefix, radixes):
    key = prefix
    if out:
        df = subs_outliers(df, df.columns, view=False)
        key += radixes["True"]
    else:
        key += radixes["False"]
    X = df.drop(columns=[y.name])
    y = df[y.name]
    return df, X, y, key, {"outliers": out}

apply_normalization_scaler = StandardScaler()
def apply_normalization(df, X, y, norm, prefix, radixes):
    key = prefix

    if norm:
        X = pd.DataFrame(apply_normalization_scaler.fit_transform(X), columns=X.columns, index=X.index)
        key += radixes["True"]
    else:
        key += radixes["False"]
    return df, X, y, key, {"normalization": norm}

def apply_pca(df, X, y, pca_n, prefix, radixes):
    key = f"{prefix}{pca_n}"
    pca_obj = PCA(n_components=pca_n)
    X_pca = pca_obj.fit_transform(X)
    return df, X_pca, y, key, {"pca": pca_n}

def build_key(key_parts, separator="_"):
    return separator.join(key_parts)

def get_preprocessing_cases(df: pd.DataFrame,
                            target: str,
                            random_seed: int,
                            feat_eng_list: list[pd.Series],
                            outliers: list[bool],
                            normalization: list[bool],
                            pca: list[int]) -> dict:
    """
    Gera múltiplos cenários de pré-processamento combinando engenharia de atributos,
    tratamento de outliers, normalização e redução de dimensionalidade (PCA).

    Cada cenário gera um conjunto `X`, `y` e um dicionário com parâmetros das transformações aplicadas.
    As chaves do dicionário final descrevem o pipeline aplicado via prefixos e sufixos codificados.

    :param df: DataFrame original contendo os dados brutos.
    :param target: Nome da coluna alvo (variável dependente) no DataFrame.
    :param random_seed: Semente para controle de aleatoriedade (reprodutibilidade).
    :param feat_eng_list: Lista de séries representando atributos derivados para engenharia de características.
                          Pode conter séries válidas ou None para ausência de feature engineering.
    :param outliers: Lista indicando se o tratamento de outliers deve ser aplicado.
    :param normalization: Lista indicando se a normalização (z-score) deve ser aplicada.
    :param pca: Lista com números de componentes principais a serem testados via PCA.

    :return: Dicionário cujas chaves são strings descritivas do pipeline (ex: 'FEna_OUT1_NORM0_PCA2'), e
             os valores são dicionários contendo:
             - "X": DataFrame com os atributos processados.
             - "y": Série da variável alvo.
             - "params": Parâmetros que descrevem as transformações aplicadas no cenário.
    """
    PREFIXES = {"feat_eng": "FE", "outliers": "OUT", "normalization": "NORM", "pca": "PCA"}
    RADIXES = {"absent": "na", "True": "1", "False": "0"}
    dict_cases = dict()
    df_original = df.copy()

    np.random.seed(random_seed) # Fixa semente aleatória

    for feat_eng, out, norm, pca_n in product(feat_eng_list, outliers, normalization, pca):
        # Etapas de pré-processamento
        param_sequence = [
            (apply_feature_engineering, feat_eng, PREFIXES["feat_eng"]),
            (apply_outliers, out, PREFIXES["outliers"]),
            (apply_normalization, norm, PREFIXES["normalization"]),
            (apply_pca, pca_n, PREFIXES["pca"]),
        ]

        key_parts = list()
        df = df_original.copy()
        X = df.drop(columns=[target])
        y = df[target]
        dict_params = dict()

        # Executa etapas
        for func, param, prefix in param_sequence:
            df, X, y, key, update = func(df, X, y, param, prefix, RADIXES)
            key_parts.append(key)
            dict_params.update(update)

        key = build_key(key_parts)
        dict_cases[key] = {"X": X, "y": y, "params": dict_params}

    return dict_cases

## Casos de KNN

In [271]:
def get_knn_cases(dict_cases_pre: dict, metrics: list[str], k_values: list[int], classes: list[str], train_size: float, random_seed: int) -> dict:
    """
    Aplica o modelo KNN sobre múltiplos cenários de pré-processamento e combinações de hiperparâmetros.

    Para cada entrada no dicionário de cenários de pré-processamento, treina modelos KNN com diferentes
    métricas de distância e valores de k, calculando métricas de avaliação para cada combinação.

    :param dict_cases_pre: Dicionário contendo os cenários de pré-processamento gerados por `get_preprocessing_cases`.
                           Cada valor deve conter as chaves: "X" (atributos), "y" (alvo) e "params" (descrição do cenário).
    :param metrics: Lista de nomes de métricas de distância aceitas pelo KNeighborsClassifier (ex: "euclidean", "manhattan").
    :param k_values: Lista de valores inteiros representando os diferentes valores de `k` (vizinhos) a serem testados.
    :param classes: Lista com os nomes das classes presentes na variável alvo, usada para cálculo das métricas por classe.
    :param train_size: Proporção dos dados a serem usados para treino.
    :param random_seed: Semente para controle de aleatoriedade (reprodutibilidade).

    :return: Dicionário em que cada chave representa um cenário e configuração de KNN (ex: "FEna_OUT1_NORM0_PCA2_KNN_METeuclidean_K3"),
             e o valor é um dicionário contendo:
             - "X": DataFrame de atributos usados no treino.
             - "y": Série com os rótulos verdadeiros.
             - "params": Dicionário com os parâmetros do cenário de pré-processamento e os hiperparâmetros do KNN.
             - "metrics": Dicionário com as métricas de avaliação geradas pela função `get_metrics_classification`.
    """
    results = dict()

    for base_key, data in dict_cases_pre.items():
        X = data["X"]
        y = data["y"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_size, random_state=random_seed)

        for metric, k in product(metrics, k_values):
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)

            metric_dict = get_metrics_classification(y_test, y_pred, classes, view=False)
            new_key = f"{base_key}_KNN_MET{metric}_K{k}"

            results[new_key] = {
                "X": X,
                "y": y,
                "params": {**data["params"], "knn_metric": metric, "knn_k": k},
                "metrics": metric_dict
            }

    return results

## Casos de Árvore de Decisão

In [281]:
def get_dtree_cases(dict_cases_pre: dict, criterions: list[str], max_depths: list[int], classes: list[str], train_size: float, random_seed: int) -> dict:
    results = dict()

    for base_key, data in dict_cases_pre.items():
        X = data["X"]
        y = data["y"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_size, random_state=random_seed)

        for depth, criterion in product(max_depths, criterions):
            dtree = DecisionTreeClassifier(criterion=criterion, max_depth=depth)
            dtree.fit(X_train, y_train)
            y_pred = dtree.predict(X_test)

            metric_dict = get_metrics_classification(y_test, y_pred, classes, view=False)
            new_key = f"{base_key}_DTREE_CRIT{criterion}_MAXDEPTH{depth}"

            results[new_key] = {
                "X": X,
                "y": y,
                "params": {**data["params"], "dtree_criterion": criterion, "dtree_max_depth": depth},
                "metrics": metric_dict
            }

    return results

## Casos de Regressão Logística

In [285]:
def get_logist_cases(dict_cases_pre: dict, penalties: list[int], max_iters: list[str], classes: list[str], train_size: float, random_seed: int) -> dict:
    SOLVER = "saga"
    results = dict()

    for base_key, data in dict_cases_pre.items():
        X = data["X"]
        y = data["y"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_size, random_state=random_seed)

        for penalty, max_iter in product(penalties, max_iters):
            logist = LogisticRegression(penalty=penalty, max_iter=max_iter, solver=SOLVER)
            logist.fit(X_train, y_train)
            y_pred = logist.predict(X_test)

            metric_dict = get_metrics_classification(y_test, y_pred, classes, view=False)
            new_key = f"{base_key}_LOGIST_PEN{penalty}_ITER{max_iter}"

            results[new_key] = {
                "X": X,
                "y": y,
                "params": {**data["params"], "logist_penalty": penalty, "logist_max_iter": max_iter},
                "metrics": metric_dict
            }

    return results

# SETUP DO AMBIENTE

In [239]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OBTENÇÃO DO DATASET

In [240]:
PATH = "/content/drive/MyDrive/07_per_shared/datasets/dataset_obesity-2024_2.csv"
df = pd.read_csv(PATH)
df_show_head(df)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,TW1
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal Weight,42.0
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal Weight,42.0
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal Weight,46.0
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Normal Weight,54.0
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Normal Weight,44.0


Shape: (2111, 18)


# EXIBIÇÃO DO DOMÍNIO

In [241]:
df_show_domain(df, df.columns)

Coluna:   Gender
dtype:    object
Domínio:  ['Female' 'Male']

Coluna:   Age
dtype:    float64
Domínio:  [21.       23.       27.       ... 22.524036 24.361936 23.664709]

Coluna:   Height
dtype:    float64
Domínio:  [1.62     1.52     1.8      ... 1.752206 1.73945  1.738836]

Coluna:   Weight
dtype:    float64
Domínio:  [ 64.        56.        77.       ... 133.689352 133.346641 133.472641]

Coluna:   family_history_with_overweight
dtype:    object
Domínio:  ['yes' 'no']

Coluna:   FAVC
dtype:    object
Domínio:  ['no' 'yes']

Coluna:   FCVC
dtype:    float64
Domínio:  [2.       3.       1.       2.450218 2.880161 2.00876  2.596579 2.591439
 2.392665 1.123939 2.027574 2.658112 2.88626  2.714447 2.750715 1.4925
 2.205439 2.059138 2.310423 2.823179 2.052932 2.596364 2.767731 2.815157
 2.737762 2.568063 2.524428 2.971574 1.0816   1.270448 1.344854 2.959658
 2.725282 2.844607 2.44004  2.432302 2.592247 2.449267 2.929889 2.015258
 1.031149 1.592183 1.21498  1.522001 2.703436 2.362918 2.140

# DADOS FALTANTES

In [242]:
print("------ ANTES DA IMPUTAÇÃO ------")
df_show_null(df)

"""
Substituamos os dados faltantes ao imputarmos a média, método
simples e escalável.
"""
NAS = df.columns[df.isnull().any()] # Colunas com ao menos um valor nulo
df_naless = subs_na_mean(df, NAS)

print("\n------ DEPOIS DA IMPUTAÇÃO ------")
df_show_null(df_naless)

------ ANTES DA IMPUTAÇÃO ------


,0
Gender,0
Age,0
Height,100
Weight,0
family_history_with_overweight,0
FAVC,0
FCVC,0
NCP,0
CAEC,0
SMOKE,0



------ DEPOIS DA IMPUTAÇÃO ------


,0
Gender,0
Age,0
Height,0
Weight,0
family_history_with_overweight,0
FAVC,0
FCVC,0
NCP,0
CAEC,0
SMOKE,0


# CODIFICAÇÃO

In [243]:
# Codificação Real
ORDINALS = ["CAEC", "CALC"]
ORDINAL_MAPS = [
    {"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3},
    {"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3},
]
df_ord = preprocessing_map(df_naless, ORDINALS, ORDINAL_MAPS)

# Codificação Binária
BINARIES = ["Gender", "family_history_with_overweight", "FAVC", "SMOKE", "SCC", "NObeyesdad"]
BINARY_MAPS = [
    {"Male": 0, "Female": 1},
    {"no": 0, "yes": 1},
    {"no": 0, "yes": 1},
    {"no": 0, "yes": 1},
    {"no": 0, "yes": 1},
    {"Normal Weight": 0, "Obesity": 1}
]
df_bin = preprocessing_map(df_ord, BINARIES, BINARY_MAPS)

# Codificação One-Hot
NOMINALS = ["MTRANS"]
df_nom = pd.get_dummies(df_bin, columns=NOMINALS, dtype="Int32")

df_show_head(df_nom)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,FAF,TUE,CALC,NObeyesdad,TW1,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,1,21.0,1.62,64.0,1,0,2.0,3.0,1,0,...,0.0,1.0,0,0,42.0,0,0,0,1,0
1,1,21.0,1.52,56.0,1,0,3.0,3.0,1,1,...,3.0,0.0,1,0,42.0,0,0,0,1,0
2,0,23.0,1.80,77.0,1,0,2.0,3.0,1,0,...,2.0,1.0,2,0,46.0,0,0,0,1,0
3,0,27.0,1.80,87.0,0,0,3.0,3.0,1,0,...,2.0,0.0,2,0,54.0,0,0,0,0,1
4,0,22.0,1.78,89.8,0,0,2.0,1.0,1,0,...,0.0,0.0,1,0,44.0,0,0,0,1,0


Shape: (2111, 22)


# ENGENHARIA DE CARACTERÍSTICAS

In [244]:
# CRIAÇÃO DA SÉRIE DO IMC (Índice de Massa Corporal)

"""
Supõe altura em metros e peso em quilogramas.
"""
se_imc = df_naless["Weight"] / (df_naless["Height"]) ** 2.0
se_imc.name = "imc"
display(se_imc)

,imc
0,24.386526
1,24.238227
2,23.765432
3,26.851852
4,28.342381
...,...
2106,44.901475
2107,43.741923
2108,43.543817
2109,44.071535


# CASOS DE PRÉ-PROCESSAMENTO

In [264]:
# Parâmetros auxiliares (mude se desejar)
TARGET = "NObeyesdad"
RANDOM_SEED = 42
FEATURE_ENGINEERING = [None, se_imc]
OUTLIERS = [False, True]
NORMALIZATION = [False, True]
PCA_LIST = [2, 4, 6, 8, 10, 12]

# Obtém casos de pré-processamento
DICT_CASES_PRE = get_preprocessing_cases(df_nom,
                        TARGET,
                        RANDOM_SEED,
                        FEATURE_ENGINEERING,
                        OUTLIERS,
                        NORMALIZATION,
                        PCA_LIST
                        )
DF_CASES_PRE = pd.DataFrame(data=DICT_CASES_PRE).T

df_show_head(DF_CASES_PRE)

,X,y,params
FEna_OUT0_NORM0_PCA2,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali..."
FEna_OUT0_NORM0_PCA4,"[[-23.444787563659887, -3.935936743582147, -0....",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali..."
FEna_OUT0_NORM0_PCA6,"[[-23.444787563659887, -3.935936743582147, -0....",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali..."
FEna_OUT0_NORM0_PCA8,"[[-23.444787563659887, -3.935936743582147, -0....",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali..."
FEna_OUT0_NORM0_PCA10,"[[-23.444787563659887, -3.935936743582147, -0....",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali..."


Shape: (48, 3)


## CASOS DE KNN

In [273]:
# Parâmetros auxiliares (mude se desejar)
METRICS = ["euclidean", "manhattan", "cosine"]
K_VALUES = [k for k in range(1, 5, 2)]
CLASSES = ["Normal Weight", "Obesity"]
TRAIN_SIZE = 0.30

# Obtém casos de KNN
DICT_CASES_KNN = get_knn_cases(DICT_CASES_PRE,
                               METRICS,
                               K_VALUES,
                               CLASSES,
                               TRAIN_SIZE,
                               RANDOM_SEED
                               )
DF_CASES_KNN = pd.DataFrame(data=DICT_CASES_KNN).T

df_show_head(DF_CASES_KNN)

,X,y,params,metrics
FEna_OUT0_NORM0_PCA2_KNN_METeuclidean_K1,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9542586750788643, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_KNN_METeuclidean_K3,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9574132492113565, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_KNN_METmanhattan_K1,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9574132492113565, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_KNN_METmanhattan_K3,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9637223974763407, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_KNN_METcosine_K1,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.8738170347003155, 'Precisão_Nor..."


Shape: (288, 4)


# CASOS DE ÁRVORE DE DECISÃO

In [277]:
# Parâmetros auxiliares (mude se desejar)
CRITERIONS = ["gini", "entropy", "log_loss"]
DEPTHS = [depth for depth in range(1, 5, 2)]

# Obtém casos de Árvore de Decisão (Dtree)
DICT_CASES_DTREE = get_dtree_cases(DICT_CASES_PRE,
                                   CRITERIONS,
                                   DEPTHS,
                                   CLASSES,
                                   TRAIN_SIZE,
                                   RANDOM_SEED
                               )
DF_CASES_DTREE = pd.DataFrame(data=DICT_CASES_DTREE).T

df_show_head(DF_CASES_DTREE)

,X,y,params,metrics
FEna_OUT0_NORM0_PCA2_DTREE_MAXDEPTH1_CRITgini,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9100946372239748, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_DTREE_MAXDEPTH1_CRITentropy,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.8990536277602523, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_DTREE_MAXDEPTH1_CRITlog_loss,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.8990536277602523, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_DTREE_MAXDEPTH3_CRITgini,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9242902208201893, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_DTREE_MAXDEPTH3_CRITentropy,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9069400630914827, 'Precisão_Nor..."


Shape: (288, 4)


# CASOS DE REGRESSÃO LOGÍSTICA

In [290]:
# Parâmetros auxiliares (mude se desejar)
PENALTIES = ["l1", "l2", None]
MAX_ITERS = [iter for iter in range(100, 2000, 100)]

# Obtém casos de Regressão Logística
DICT_CASES_LOGIST = get_logist_cases(DICT_CASES_PRE,
                                   PENALTIES,
                                   MAX_ITERS,
                                   CLASSES,
                                   TRAIN_SIZE,
                                   RANDOM_SEED
                               )
DF_CASES_LOGIST = pd.DataFrame(data=DICT_CASES_LOGIST).T

df_show_head(DF_CASES_LOGIST)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which 

,X,y,params,metrics
FEna_OUT0_NORM0_PCA2_LOGIST_PENl1_ITER100,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9100946372239748, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_LOGIST_PENl1_ITER200,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9132492113564669, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_LOGIST_PENl1_ITER300,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9100946372239748, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_LOGIST_PENl1_ITER400,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9100946372239748, 'Precisão_Nor..."
FEna_OUT0_NORM0_PCA2_LOGIST_PENl1_ITER500,"[[-23.4447875636599, -3.935936743582154], [-31...",0 0 1 0 2 0 3 0 4 ...,"{'feat_eng': None, 'outliers': False, 'normali...","{'Acurácia': 0.9100946372239748, 'Precisão_Nor..."


Shape: (2736, 4)
